In [1]:
import pandas as pd
import numpy as np
import random

In [209]:
groups = ['Queen', 'Led Zeppelin', 'Black Sabbath', 'Deep Purple', 'Scorpions',
          'Metallica','Iron Maiden',  'Megadeth', 'Judas Priest', 'Motorhead',
          'Kendrick Lamar', 'Tyler, the Creator', 'Kanye West', '2pac', 'Jay-Z',
          'Radiohead', 'Bjork', 'My Bloody Valentine', 'Madvillain', 'Godspeed You Black Emperor!',
          'Michael Jackson', 'Madonna', 'George Michael', 'Janet Jackson', 'Prince']

In [210]:
groups = groups[:5] + groups[10:15]
group_df = pd.DataFrame(index=groups, columns=groups)
popularities = []
for i in range(len(groups) // 5):
    popularities.extend(range(5, 0, -1))


for i in range(len(groups) // 5):
    for j in range(5 * i):
        for k in range(5 * i, 5 * i + 5):
            group_df.iloc[j, k] = group_df.iloc[k, j] = 1
    for j in range(5 * i, 5 * i + 5):
        for k in range(5 * i, 5 * i + 5):
            if j != k:
                group_df.iloc[j, k] = group_df.iloc[k, j] = 3
            else:
                group_df.iloc[j, k] = 0
    for j in range(5 * i + 5, len(groups)):
        for k in range(5 * i, 5 * i + 5):
            group_df.iloc[j, k] = group_df.iloc[k, j] = 3

for i in range(len(groups)):
    group_df.iloc[i] *= popularities[i]
group_df

,Queen,Led Zeppelin,Black Sabbath,Deep Purple,Scorpions,Kendrick Lamar,"Tyler, the Creator",Kanye West,2pac,Jay-Z
Queen,0,15,15,15,15,5,5,5,5,5
Led Zeppelin,12,0,12,12,12,4,4,4,4,4
Black Sabbath,9,9,0,9,9,3,3,3,3,3
Deep Purple,6,6,6,0,6,2,2,2,2,2
Scorpions,3,3,3,3,0,1,1,1,1,1
Kendrick Lamar,5,5,5,5,5,0,15,15,15,15
"Tyler, the Creator",4,4,4,4,4,12,0,12,12,12
Kanye West,3,3,3,3,3,9,9,0,9,9
2pac,2,2,2,2,2,6,6,6,0,6
Jay-Z,1,1,1,1,1,3,3,3,3,0


In [211]:
transaction_count = 50_000
transactions = {'Transaction': [], 'Item': []}
for tr_id in range(transaction_count):
    items = random.choices(groups, weights=popularities)
    item_count = random.randint(1, 4)
    new_items = set(random.choices(groups, weights=group_df[items[0]], k=item_count - 1)).difference(set(items))
    items.extend(list(new_items))
    for item in items:
        transactions['Transaction'].append(tr_id + 1)
        transactions['Item'].append(item)
transactions = pd.DataFrame(transactions)
transactions

,Transaction,Item
0,1,Deep Purple
1,1,Black Sabbath
2,1,Led Zeppelin
3,2,Kendrick Lamar
4,2,Kanye West
...,...,...
117030,49999,Kendrick Lamar
117031,50000,"Tyler, the Creator"
117032,50000,Scorpions
117033,50000,Queen


In [ ]:
transactions.to_csv('groups-long.csv', index=False, encoding='utf-8')

# Анализ датасета

In [213]:
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

In [247]:
df = pd.read_csv("./groups-short.csv", encoding='utf-8')

In [248]:
df.head(100)

,Transaction,Item
0,1,Queen
1,1,Black Sabbath
2,1,Deep Purple
3,2,Queen
4,3,Queen
5,3,Led Zeppelin
6,3,Black Sabbath
7,4,Queen
8,4,Scorpions
9,5,Led Zeppelin


In [249]:
df

,Transaction,Item
0,1,Queen
1,1,Black Sabbath
2,1,Deep Purple
3,2,Queen
4,3,Queen
5,3,Led Zeppelin
6,3,Black Sabbath
7,4,Queen
8,4,Scorpions
9,5,Led Zeppelin


In [250]:
df['Item'] = df['Item'].apply(lambda x: x.lower().strip())
df['Item'] = df.loc[df['Item'] != 'none', ['Item']]
df = df.dropna()

In [251]:
len(df['Item'].unique())

5

In [252]:
df['Transaction'].max() == len(df['Transaction'].unique())

np.True_

In [253]:
unique_items = df['Item'].unique()
_dict = defaultdict(list)
for transaction_number in tqdm(df['Transaction'].unique()):
    products = df.loc[df['Transaction'] == transaction_number]['Item'].to_list()
    for pr in unique_items:
        if pr in products:
            _dict[pr].append(1)
        else:
            _dict[pr].append(0)
norm_df = pd.DataFrame(_dict)
norm_df

100%|██████████| 15/15 [00:00<00:00, 2998.79it/s]


,queen,black sabbath,deep purple,led zeppelin,scorpions
0,1,1,1,0,0
1,1,0,0,0,0
2,1,1,0,1,0
3,1,0,0,0,1
4,0,0,0,1,0
5,1,0,1,0,1
6,1,0,0,1,0
7,1,0,1,1,0
8,1,1,1,1,0
9,0,0,0,1,0


In [254]:
norm_df.sum()

queen            12
black sabbath     7
deep purple       5
led zeppelin      9
scorpions         4
dtype: int64

# Алгоритм Apriori

In [255]:
from itertools import combinations, permutations
from tqdm import tqdm

In [256]:
def count_rows(items: list[str]):
    condition = (norm_df[items[0]] == 1)
    for i in range(1, len(items)):
        condition &= (norm_df[items[i]] == 1)
    cnt = len(norm_df[condition])
    return cnt

In [257]:
k = 1
f = 4
results: list[list[set]] = []
while True:
    f_k: list[set] = []
    if k == 1:
        for item in unique_items:
            if norm_df.sum()[item] >= f:
                f_k.append({item})
    else:
        if k == 2:
            combs: list[set] = [x[0].union(x[1]) for x in combinations(results[0], r=2)]
        else:
            previous = results[k - 2]
            combs = []
            for i, elem in tqdm(enumerate(previous)):
                _set = set(sorted(elem)[:-1])
                if any(_set.issubset(x) for x in combs):
                    continue
                for j, elem1 in enumerate(previous):
                    if i == j:
                        continue
                    if _set.issubset(elem1) and all(any(len(set(subset).difference(x)) == 0 for x in previous) for subset in combinations(elem.union(elem1), r=k-1)):
                        combs.append(elem.union(elem1))
            
            # combs = []
            # for elem in possible:
            #     flag = True
            #     for subset in combinations(elem, r=k-1):
            #         cnt = count_rows(subset)
            #         if cnt < f:
            #             flag = False
            #             break
            #     if flag:
            #         combs.append(elem)
        
        for elem in combs:
            cnt = count_rows(list(elem))
            if cnt >= f:
                f_k.append(elem)
    
    if len(f_k) == 0:
        break
    results.append(f_k)
    k += 1

4it [00:00, ?it/s]
1it [00:00, ?it/s]


In [258]:
results

[[{'queen'},
  {'black sabbath'},
  {'deep purple'},
  {'led zeppelin'},
  {'scorpions'}],
 [{'black sabbath', 'queen'},
  {'deep purple', 'queen'},
  {'led zeppelin', 'queen'},
  {'black sabbath', 'led zeppelin'}],
 [{'black sabbath', 'led zeppelin', 'queen'}]]

In [259]:
results.pop(0)
while type(results[0]) != set:
    for elem in results[0]:
        results.append(elem)
    results.pop(0)
results

[{'black sabbath', 'queen'},
 {'deep purple', 'queen'},
 {'led zeppelin', 'queen'},
 {'black sabbath', 'led zeppelin'},
 {'black sabbath', 'led zeppelin', 'queen'}]

In [262]:
rules = pd.DataFrame(columns=['Условие', 'Следствие', 'Поддержка', 'Достоверность', 'Убеждённость', 'Лифт', 'Рычаг'])
# rules = {'Условие': [], 'Следствие': [], 'Поддержка': [], 'Достоверность': []}
for elem in results:
    for p in combinations(elem, r=len(elem)-1):
        condition = list(p)
        consequence = list(set(elem).difference(set(p)))
        support = count_rows(condition + consequence) / len(norm_df)
        confidence = count_rows(condition + consequence) / count_rows(condition)
        conviction = ((1 - count_rows(consequence) / len(norm_df)) / (1 - confidence) if confidence != 1 else np.inf)
        lift = (count_rows(consequence + condition) * len(norm_df)) / (count_rows(condition) * count_rows(consequence))
        leverage = (count_rows(consequence + condition) * len(norm_df) - count_rows(condition) * count_rows(consequence)) / len(norm_df) ** 2
        rules.loc[len(rules)] = [condition, consequence, support * 100, confidence * 100, conviction * 100, lift, leverage]
rules

,Условие,Следствие,Поддержка,Достоверность,Убеждённость,Лифт,Рычаг
0,[black sabbath],[queen],40.000000,85.714286,140.000000,1.071429,0.026667
1,[queen],[black sabbath],40.000000,50.000000,106.666667,1.071429,0.026667
2,[deep purple],[queen],33.333333,100.000000,inf,1.250000,0.066667
3,[queen],[deep purple],33.333333,41.666667,114.285714,1.250000,0.066667
4,[led zeppelin],[queen],46.666667,77.777778,90.000000,0.972222,-0.013333
5,[queen],[led zeppelin],46.666667,58.333333,96.000000,0.972222,-0.013333
6,[led zeppelin],[black sabbath],26.666667,44.444444,96.000000,0.952381,-0.013333
7,[black sabbath],[led zeppelin],26.666667,57.142857,93.333333,0.952381,-0.013333
8,"[led zeppelin, black sabbath]",[queen],26.666667,100.000000,inf,1.250000,0.053333
9,"[led zeppelin, queen]",[black sabbath],26.666667,57.142857,124.444444,1.224490,0.048889


In [263]:
rules['Поддержка'].max()

np.float64(46.666666666666664)

In [264]:
rules['Достоверность'].max()

np.float64(100.0)

In [265]:
rules[(rules['Достоверность'] >= 30)].sort_values('Достоверность')

,Условие,Следствие,Поддержка,Достоверность,Убеждённость,Лифт,Рычаг
3,[queen],[deep purple],33.333333,41.666667,114.285714,1.250000,0.066667
6,[led zeppelin],[black sabbath],26.666667,44.444444,96.000000,0.952381,-0.013333
1,[queen],[black sabbath],40.000000,50.000000,106.666667,1.071429,0.026667
7,[black sabbath],[led zeppelin],26.666667,57.142857,93.333333,0.952381,-0.013333
9,"[led zeppelin, queen]",[black sabbath],26.666667,57.142857,124.444444,1.224490,0.048889
5,[queen],[led zeppelin],46.666667,58.333333,96.000000,0.972222,-0.013333
10,"[black sabbath, queen]",[led zeppelin],26.666667,66.666667,120.000000,1.111111,0.026667
4,[led zeppelin],[queen],46.666667,77.777778,90.000000,0.972222,-0.013333
0,[black sabbath],[queen],40.000000,85.714286,140.000000,1.071429,0.026667
2,[deep purple],[queen],33.333333,100.000000,inf,1.250000,0.066667


# Алгоритм Eclat

In [231]:
eclat_dict = {'Item': [], 'Transactions': []}
min_support = 4
for item in sorted(unique_items):
    transactions = norm_df[norm_df[item] == 1].index.to_list()
    eclat_dict['Item'].append([item])
    eclat_dict['Transactions'].append(transactions)
eclat_df = pd.DataFrame(eclat_dict)
eclat_df = eclat_df[eclat_df['Transactions'].apply(lambda t: len(t) >= min_support)]
eclat_df

,Item,Transactions
0,[2pac],"[2, 5, 6, 11, 14, 15, 16, 19, 32, 35, 45, 46, ..."
1,[black sabbath],"[0, 10, 12, 18, 23, 26, 27, 37, 47, 50, 52, 53..."
2,[deep purple],"[0, 13, 17, 18, 20, 22, 26, 33, 34, 36, 38, 48..."
3,[jay-z],"[4, 10, 31, 62, 90, 97, 108, 135, 139, 150, 15..."
4,[kanye west],"[1, 6, 9, 10, 15, 17, 23, 27, 29, 30, 33, 35, ..."
5,[kendrick lamar],"[1, 2, 3, 4, 5, 6, 8, 10, 11, 13, 16, 19, 20, ..."
6,[led zeppelin],"[0, 3, 12, 13, 15, 17, 21, 25, 27, 36, 37, 38,..."
7,[queen],"[3, 7, 12, 19, 20, 21, 24, 26, 27, 34, 36, 37,..."
8,[scorpions],"[1, 5, 6, 38, 50, 54, 65, 80, 90, 91, 101, 103..."
9,"[tyler, the creator]","[2, 3, 4, 7, 8, 11, 15, 19, 23, 28, 29, 31, 33..."


In [232]:
min_support = 4
start_point = 0
k = 2
while True:
    end_point = len(eclat_df)
    
    new_dict = {'Item': [], 'Transactions': []}
    for i in tqdm(range(start_point, end_point)):
        for j in range(i + 1, end_point):
            item1, item2 = eclat_df.iloc[i]['Item'], eclat_df.iloc[j]['Item']
            if k > 2 and not all(x == y for x in item1[:k - 2] for y in item2[:k - 2]):
                break

            tr1, tr2 = eclat_df.iloc[i]['Transactions'], eclat_df.iloc[j]['Transactions']
            if len(set(item1).union(set(item2))) == k:
                new_dict['Item'].append(sorted(set(item1).union(set(item2))))
                new_dict['Transactions'].append(list(set(tr1).intersection(set(tr2))))
    
    eclat_df = pd.concat([
        eclat_df,
        pd.DataFrame(new_dict)
    ], ignore_index=True)
    if len(eclat_df) == end_point:
        break
    k += 1
    start_point = end_point
    eclat_df = eclat_df[eclat_df['Transactions'].apply(lambda t: len(t) >= min_support)]

eclat_df

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 120/120 [00:00<00:00, 15000.64it/s]


,Item,Transactions
0,[2pac],"[2, 5, 6, 11, 14, 15, 16, 19, 32, 35, 45, 46, ..."
1,[black sabbath],"[0, 10, 12, 18, 23, 26, 27, 37, 47, 50, 52, 53..."
2,[deep purple],"[0, 13, 17, 18, 20, 22, 26, 33, 34, 36, 38, 48..."
3,[jay-z],"[4, 10, 31, 62, 90, 97, 108, 135, 139, 150, 15..."
4,[kanye west],"[1, 6, 9, 10, 15, 17, 23, 27, 29, 30, 33, 35, ..."
...,...,...
170,"[kendrick lamar, scorpions, tyler, the creator]","[16384, 28160, 23043, 43015, 36361, 18956, 871..."
171,"[led zeppelin, queen, scorpions]","[36864, 32771, 49162, 16394, 4107, 24590, 2065..."
172,"[led zeppelin, queen, tyler, the creator]","[2048, 43009, 3, 49155, 47110, 14344, 32779, 4..."
173,"[led zeppelin, scorpions, tyler, the creator]","[31747, 6148, 23050, 4107, 41998, 35856, 22546..."


In [243]:
eclat_df = eclat_df[eclat_df['Item'].apply(lambda x: len(x) > 1)]

In [244]:
rules = pd.DataFrame(columns=['Условие', 'Следствие', 'Поддержка', 'Достоверность', 'Убеждённость', 'Лифт', 'Рычаг'])
# rules = {'Условие': [], 'Следствие': [], 'Поддержка': [], 'Достоверность': []}
for elem in eclat_df['Item']:
    for p in combinations(elem, r=len(elem)-1):
        condition = list(p)
        consequence = list(set(elem).difference(set(p)))
        support = count_rows(condition + consequence) / len(norm_df)
        confidence = count_rows(condition + consequence) / count_rows(condition)
        conviction = (1 - count_rows(consequence) / len(norm_df)) / (1 - confidence)
        lift = (count_rows(consequence + condition) * len(norm_df)) / (count_rows(condition) * count_rows(consequence))
        leverage = (count_rows(consequence + condition) * len(norm_df) - count_rows(condition) * count_rows(consequence)) / len(norm_df) ** 2
        rules.loc[len(rules)] = [condition, consequence, support * 100, confidence * 100, conviction * 100, lift, leverage]
rules

,Условие,Следствие,Поддержка,Достоверность,Убеждённость,Лифт,Рычаг
0,[2pac],[black sabbath],2.426,14.192114,88.367170,0.587082,-0.017063
1,[black sabbath],[2pac],2.426,10.035575,92.154205,0.587082,-0.017063
2,[2pac],[deep purple],1.454,8.505909,90.869256,0.504502,-0.014280
3,[deep purple],[2pac],1.454,8.623962,90.730570,0.504502,-0.014280
4,[2pac],[jay-z],1.734,10.143910,101.384336,1.139765,0.002126
...,...,...,...,...,...,...,...
445,"[led zeppelin, tyler, the creator]",[scorpions],0.344,5.966008,96.677805,0.656326,-0.001801
446,"[scorpions, tyler, the creator]",[led zeppelin],0.344,22.811671,89.964378,0.746504,-0.001168
447,"[queen, scorpions]","[tyler, the creator]",0.412,10.651499,77.516690,0.346503,-0.007770
448,"[queen, tyler, the creator]",[scorpions],0.412,5.924648,96.635301,0.651776,-0.002201


In [235]:
rules['Поддержка'].max()

np.float64(13.774000000000001)

In [245]:
rules['Достоверность'].max()

np.float64(46.476063829787236)

In [246]:
rules[(rules['Достоверность'] >= 30)].sort_values('Достоверность')

,Условие,Следствие,Поддержка,Достоверность,Убеждённость,Лифт,Рычаг
277,"[black sabbath, tyler, the creator]",[queen],1.318,30.022779,91.438327,0.833642,-0.002630
243,"[black sabbath, kanye west]",[queen],1.018,30.154028,91.610151,0.837286,-0.001978
364,"[jay-z, queen]",[kendrick lamar],0.550,30.186608,91.426585,0.834530,-0.001091
397,"[kanye west, scorpions]",[kendrick lamar],0.378,30.288462,91.560166,0.837346,-0.000734
401,"[kendrick lamar, tyler, the creator]",[kanye west],4.194,30.448671,108.598932,1.244428,0.008238
442,"[led zeppelin, tyler, the creator]",[queen],1.780,30.870621,92.559778,0.857184,-0.002966
420,"[kendrick lamar, led zeppelin]",[queen],2.220,31.118587,92.892984,0.864069,-0.003492
437,"[scorpions, tyler, the creator]",[kendrick lamar],0.470,31.167109,92.728925,0.861636,-0.000755
391,"[kanye west, led zeppelin]",[kendrick lamar],1.390,31.207903,92.783914,0.862764,-0.002211
394,"[kanye west, queen]",[kendrick lamar],1.680,31.215162,92.793705,0.862965,-0.002668
